In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
import torchvision.models as models

In [24]:
BATCH_SIZE = 16
LEARNING_RATE = 0.0001
NUM_WORKERS = 1
NUM_EPOCHS = 100
np.random.seed(42)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [25]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder('./sdnet-data/pavement', transform=transform)

classes = ('Cracked', 'Not Cracked')

In [26]:
"""
Split: 70% train | 15% validation | 15% test
"""
dataset_size = len(dataset)
train_split = int(np.floor(0.70 * dataset_size))
val_split = train_split + int(np.floor(0.15 * dataset_size))

indices = list(range(dataset_size))
np.random.shuffle(indices)
train_idxs, val_idxs, test_idxs = indices[:train_split], indices[train_split:val_split], indices[val_split:]


train_sampler = SubsetRandomSampler(train_idxs)
val_sampler = SubsetRandomSampler(val_idxs)
test_sampler = SubsetRandomSampler(test_idxs)

In [27]:
"""
Load data
"""

trainloader = DataLoader(
    dataset, batch_size=BATCH_SIZE, sampler=train_sampler,
    num_workers=NUM_WORKERS,
)

valloader = DataLoader(
    dataset, batch_size=BATCH_SIZE, sampler=val_sampler,
    num_workers=NUM_WORKERS,
)

testloader = DataLoader(
    dataset, batch_size=BATCH_SIZE, sampler=test_sampler,
    num_workers=NUM_WORKERS,
)

In [28]:
"""
Examine class balances in splits
"""

def view_class_balance(dataloader):
    total = 0
    not_cracked_total = 0
    for data in dataloader:
        images, labels = data
        total += len(labels)
        not_cracked_total += sum(labels).item()

    print("There are %i examples total" % total)
    print("%i or %.2f%% are not cracked" % (not_cracked_total, (not_cracked_total/total)*100))
    print("%i or %.2f%% are cracked" % (total - not_cracked_total, ((total - not_cracked_total)/total)*100))
    print()
    
view_class_balance(trainloader)
view_class_balance(valloader)
view_class_balance(testloader)

There are 3651 examples total
1806 or 49.47% are not cracked
1845 or 50.53% are cracked

There are 782 examples total
391 or 50.00% are not cracked
391 or 50.00% are cracked

There are 783 examples total
411 or 52.49% are not cracked
372 or 47.51% are cracked



In [29]:
"""
Helper functions to view images.


def imshow(img):
    img = img / 2 + 0.5     # unnormalize - corresponds to transformation above.
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(testloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[0]))
print('Labeled as:', classes[labels[0].item()])
"""
h = models.resnet18()
h.fc = nn.Linear(512, 2)
h.conv1
print(h.conv1)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [30]:

"""
Define Model
"""

class DetectionModel(nn.Module):
    
    def __init__(self):
        super(DetectionModel, self).__init__()
        self.resnet = models.resnet18()
        self.resnet.fc = nn.Linear(512, 2)

        
    def forward(self, x):
        x = self.resnet(x)
        x = F.log_softmax(x, dim=1)
        
        return x

In [31]:
"""
Instantiate a MNISTModel to be trained
"""

model = DetectionModel().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [32]:
"""
Train model
"""

for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        optimizer.zero_grad()

        preds = model(inputs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('Epoch: %d | loss: %.3f' % (epoch + 1, running_loss / len(trainloader)))

Epoch: 1 | loss: 0.583
Epoch: 2 | loss: 0.485
Epoch: 3 | loss: 0.426
Epoch: 4 | loss: 0.346
Epoch: 5 | loss: 0.245
Epoch: 6 | loss: 0.193
Epoch: 7 | loss: 0.154
Epoch: 8 | loss: 0.086
Epoch: 9 | loss: 0.085
Epoch: 10 | loss: 0.123
Epoch: 11 | loss: 0.071
Epoch: 12 | loss: 0.068
Epoch: 13 | loss: 0.054
Epoch: 14 | loss: 0.042
Epoch: 15 | loss: 0.051
Epoch: 16 | loss: 0.095
Epoch: 17 | loss: 0.041
Epoch: 18 | loss: 0.029
Epoch: 19 | loss: 0.036
Epoch: 20 | loss: 0.052
Epoch: 21 | loss: 0.028
Epoch: 22 | loss: 0.021
Epoch: 23 | loss: 0.023
Epoch: 24 | loss: 0.024
Epoch: 25 | loss: 0.051
Epoch: 26 | loss: 0.029
Epoch: 27 | loss: 0.019
Epoch: 28 | loss: 0.071
Epoch: 29 | loss: 0.020
Epoch: 30 | loss: 0.043
Epoch: 31 | loss: 0.025
Epoch: 32 | loss: 0.013
Epoch: 33 | loss: 0.011
Epoch: 34 | loss: 0.022
Epoch: 35 | loss: 0.093
Epoch: 36 | loss: 0.022
Epoch: 37 | loss: 0.015
Epoch: 38 | loss: 0.017
Epoch: 39 | loss: 0.018
Epoch: 40 | loss: 0.052
Epoch: 41 | loss: 0.006
Epoch: 42 | loss: 0.002
E

In [33]:
"""
Test model
"""

correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        inputs, targets = data
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        preds = model(inputs)
        _, predicted = torch.max(preds.data, 1) # max of logits
        
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 77 %
